In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __init__ import *
from new_solutions.simple_problem import study_many_problems
from runner.svg2plan import run_new_layout
from new_solutions.selection import *
from itertools import accumulate
from operator import add 
from functools import reduce
from visuals.plotter import plot_general
# from unit_tests.test_solutions import *
from typing import Literal, List
from enum import Enum
from collections import Counter
from typing import TypeVar
from typing import Any
from domains.domain import Domain

import plotly.io as pio
pio.renderers.default = "browser"

In [3]:
report = run_new_layout()
c = Cook(report)


executer> 
 begining to execute stacking
helloo!
skipping bc of corner overlap 0
skipping bc of corner overlap 1
skipping bc of corner overlap 2
skipping bc of corner overlap 3
skipping bc of corner overlap 4
skipping bc of corner overlap 5
skipping bc of corner overlap 6
skipping bc of corner overlap 7
first bl to try -> node: laundry, action: STRETCH, summary: Counter({'OVERLAP': 4}), # unres probs: 4
problem this fixes -> Problem(index=4, problem_type=ProblemType.HOLE, resolved=False, nbs=['transit_space', 'laundry', 'm_bath', 'bath']
0


In [4]:
c.run_again()

running again -> 1
helloo!
first bl to try -> node: laundry, action: SQUEEZE, summary: Counter({'OVERLAP': 3}), # unres probs: 3
problem this fixes -> Problem(index=5, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_bath', 'laundry')


In [5]:
c.run_again()

running again -> 2
helloo!
first bl to try -> node: bath, action: SQUEEZE, summary: Counter({'OVERLAP': 2}), # unres probs: 2
problem this fixes -> Problem(index=2, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_bedroom', 'bath')


In [6]:
c.run_again()

running again -> 3
helloo!
first bl to try -> node: m_bedroom, action: SQUEEZE, summary: Counter({'OVERLAP': 1}), # unres probs: 1
problem this fixes -> Problem(index=3, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_bedroom', 'dining')


In [7]:
c.run_again()

running again -> 4
helloo!
first bl to try -> node: m_closet, action: SQUEEZE, summary: Counter(), # unres probs: 0
problem this fixes -> Problem(index=1, problem_type=ProblemType.OVERLAP, resolved=False, nbs=('m_closet', 'dining')


In [ ]:
c.plot(4)